<a href="https://colab.research.google.com/github/Normal-case/DeepLearning/blob/master/DNN_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.nn import relu
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
data = datasets.load_breast_cancer()
X = np.array(data.data, dtype = 'float32')
y = np.array(data.target, dtype = 'float32')

In [4]:
y = y.reshape(-1, 1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 777)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [7]:
import pandas as pd
df = pd.DataFrame(X, columns = data.feature_names)
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.990000,10.380000,122.800003,1001.000000,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.399994,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.379999,17.330000,184.600006,2019.000000,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.570000,17.770000,132.899994,1326.000000,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.080002,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990000,23.410000,158.800003,1956.000000,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.690001,21.250000,130.000000,1203.000000,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.029999,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570000,25.530001,152.500000,1709.000000,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.420000,20.379999,77.580002,386.100006,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.230000,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910000,26.500000,98.870003,567.700012,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.290001,14.340000,135.100006,1297.000000,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.440002,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540001,16.670000,152.199997,1575.000000,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.559999,22.389999,142.000000,1479.000000,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.699997,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450001,26.400000,166.100006,2027.000000,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.129999,28.250000,131.199997,1261.000000,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.040001,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690001,38.250000,155.000000,1731.000000,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.600000,28.080000,108.300003,858.099976,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.549999,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980000,34.119999,126.699997,1124.000000,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.600000,29.330000,140.100006,1265.000000,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.220001,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740000,39.419998,184.600006,1821.000000,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [8]:
df.info() # 총 칼럼이 30개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float32
 1   mean texture             569 non-null    float32
 2   mean perimeter           569 non-null    float32
 3   mean area                569 non-null    float32
 4   mean smoothness          569 non-null    float32
 5   mean compactness         569 non-null    float32
 6   mean concavity           569 non-null    float32
 7   mean concave points      569 non-null    float32
 8   mean symmetry            569 non-null    float32
 9   mean fractal dimension   569 non-null    float32
 10  radius error             569 non-null    float32
 11  texture error            569 non-null    float32
 12  perimeter error          569 non-null    float32
 13  area error               569 non-null    float32
 14  smoothness error         5

In [9]:
X_train.shape

(455, 30)

In [10]:
# w와 b를 초기화하는 객체
initializer = tf.contrib.layers.xavier_initializer()

# 30줄 30칸의 w0를 생성
# 30줄 10칸의 w1를 생성
# 10줄 1칸의 w2를 생성
w0 = tf.Variable(initializer([30, 30]))
b0 = tf.Variable(initializer([30]))
w1 = tf.Variable(initializer([30, 10]))
b1 = tf.Variable(initializer([10]))
w2 = tf.Variable(initializer([10, 1]))
b2 = tf.Variable(initializer([1]))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [11]:
# hypothesis0에서 30번의 선형회귀, hypothesis0의 결과를 통해서 총 10번의 선형회귀, hypothesis1의 결괄르 통해서 총 한번의 선형회귀 더
hypothesis0 = tf.sigmoid(tf.matmul(X_train, w0) + b0)
hypothesis1 = tf.sigmoid(tf.matmul(hypothesis0, w1) + b1)
hypothesis2 = tf.sigmoid(tf.matmul(hypothesis1, w2) + b2)

In [12]:
optimizer = tf.train.AdamOptimizer(0.01)

for step in range(1000):
  with tf.GradientTape() as tape:
    hypothesis0 = tf.sigmoid(tf.matmul(X_train, w0) + b0)
    hypothesis1 = tf.sigmoid(tf.matmul(hypothesis0, w1) + b1)
    hypothesis2 = tf.sigmoid(tf.matmul(hypothesis1, w2) + b2)
    cost = -tf.reduce_mean(y_train * tf.log(hypothesis2) + (1 - y_train) * tf.log(1 - hypothesis2))
    grads = tape.gradient(cost, [w0, w1, w2, b0, b1, b2])
    optimizer.apply_gradients(grads_and_vars=zip(grads, [w0, w1, w2, b0, b1, b2]))

    if step % 100 == 0:
      print('*'*100)
      print('step: {}\ncost :\n{}'.format(step, cost.numpy()))
      print('*'*100)

****************************************************************************************************
step: 0
cost :
0.6812898516654968
****************************************************************************************************
****************************************************************************************************
step: 100
cost :
0.03140179440379143
****************************************************************************************************
****************************************************************************************************
step: 200
cost :
0.019359968602657318
****************************************************************************************************
****************************************************************************************************
step: 300
cost :
0.016683904454112053
****************************************************************************************************
*********************************************

In [13]:
X_test = scaler.transform(X_test)
hypothesis0 = tf.sigmoid(tf.matmul(X_test, w0) + b0)
hypothesis1 = tf.sigmoid(tf.matmul(hypothesis0, w1) + b1)
predict = tf.sigmoid(tf.matmul(hypothesis1, w2) + b2)
predict = tf.cast(predict > 0.5, dtype = tf.float32)
accuracy = np.sum(tf.equal(predict, y_test)) / len(predict)
print('정확도 :', accuracy)

정확도 : 0.9385964912280702
